## CIFAR

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import uuid
import numpy as np

def extract_quality(filepath, quality_metric='f1'):
    """
    Extract quality metrics from an Excel file.
    Returns init and step metrics for the specified quality metric.
    """
    xls = pd.ExcelFile(filepath)
    quality = pd.read_excel(xls, 'quality_comparison')
    quality.columns = quality.iloc[0]
    quality = quality.drop(quality.index[0])
    init_val = quality[quality['mode'] == quality_metric]['original'].values[0]
    step_val = quality[quality['mode'] == quality_metric]['approach'].values[0]
    return float(init_val), float(step_val)

def extract_computational(filepath):
    """
    Extract computational metrics from an Excel file.
    Returns latency, size, and throughput for init and step.
    """
    xls = pd.ExcelFile(filepath)
    computational = pd.read_excel(xls, 'computational_comparison')
    latency_init, size_init, thr_init = computational.iloc[2, 1], computational.iloc[3, 1], computational.iloc[4, 1]
    latency_step, size_step, thr_step = computational.iloc[2, 2], computational.iloc[3, 2], computational.iloc[4, 2]
    return (float(latency_init), float(size_init), float(thr_init)), (float(latency_step), float(size_step), float(thr_step))

def analyze_results(gpu_path, cpu_path, tab_name, quality_metric='f1', steps = 3):
    """
    Analyze PEFT pipeline results and generate plots and tables for top 5 pipelines.
    """
    # Create output directories
    os.makedirs('imgs', exist_ok=True)
    os.makedirs('tables', exist_ok=True)

    # Initialize results storage
    results = []

    # Process both GPU and CPU folders
    for device_path, device in [(gpu_path, 'GPU'), (cpu_path, 'CPU')]:
        if device_path is None:
            continue
        for pipeline in os.listdir(device_path):
            if pipeline == '.DS_Store':
                continue
            xlsx_files = [f for f in os.listdir(os.path.join(device_path, pipeline)) if f.endswith('.xlsx')]
            count = len(xlsx_files)
            if count < steps:
                continue

            pipeline_path = os.path.join(device_path, pipeline)
            if not os.path.isdir(pipeline_path):
                continue

            quality_metrics = []
            comp_metrics = []
            for step_file in sorted(os.listdir(pipeline_path)):
                if not step_file.endswith('.xlsx'):
                    continue
                step_path = os.path.join(pipeline_path, step_file)
                quality_metrics.append(extract_quality(step_path, quality_metric))
                comp_metrics.append(extract_computational(step_path))


            # try:
            init_quality, final_quality = quality_metrics[0][0], quality_metrics[-1][1]
            (init_latency, init_size, init_thr), (final_latency, final_size, final_thr) = comp_metrics[0][0], comp_metrics[-1][1]
            # except:
            #     _ = 1


            quality_change = ((final_quality - init_quality) / init_quality * 100) if init_quality != 0 else 0
            latency_change = ((final_latency - init_latency) / init_latency * 100) if init_latency != 0 else 0
            size_change = ((final_size - init_size) / init_size * 100) if init_size != 0 else 0
            thr_change = ((final_thr - init_thr) / init_thr * 100) if init_thr != 0 else 0

            results.append({
                'pipeline': pipeline,
                'device': device,
                'quality_init': init_quality,
                'quality_final': final_quality,
                'quality_change': quality_change,
                'latency': final_latency,
                'latency_change': latency_change,
                'throughput': final_thr,
                'throughput_change': thr_change,
                'size': final_size,
                'size_change': size_change
            })

    df = pd.DataFrame(results)
    if quality_metric.lower() == 'rmse':
        top_5_cpu = df[df['device'] == 'CPU'].sort_values('quality_final').head(3)
        top_5_gpu = df[df['device'] == 'GPU'].sort_values('quality_final').head(3)
    else:
        # top_5 = df.sort_values('quality_final', ascending=False).head(5)
        top_5_cpu = df[df['device'] == 'CPU'].sort_values('quality_final', ascending=False).head(3)
        top_5_gpu = df[df['device'] == 'GPU'].sort_values('quality_final', ascending=False).head(3)

    top_5 = pd.concat([top_5_gpu, top_5_cpu], axis=0).reset_index(drop=True)
    if quality_metric.lower() == 'rmse':
        best_top = top_5.sort_values('quality_final').head(6)
    else:
        best_top = top_5.sort_values('quality_final', ascending=False).head(6)
    #  top 5 pipelines
    print("Top 6 Pipelines:")
    for _, row in best_top.iterrows():
        print(f"{row['device']} - {row['pipeline']}: {quality_metric} = {row['quality_final']:.3f} ({row['quality_change']:+.1f}%)")

    plt.figure(figsize=(12, 6))
    bar_width = 0.35
    index = np.arange(len(top_5))
    plt.bar(index, top_5['quality_init'], bar_width, label='Before')
    plt.bar(index + bar_width, top_5['quality_final'], bar_width, label='After')
    plt.xlabel('Pipeline')
    plt.ylabel(quality_metric)
    plt.title(f'{quality_metric} Before and After Pipeline')
    plt.xticks(index + bar_width/2, [f"{row['device']}: {row['pipeline']}" for _, row in top_5.iterrows()], rotation=45, ha='right')
    plt.legend()
    plt.tight_layout()
    # plt.savefig('imgs/quality_comparison.png')
    plt.close()

    #  percentage changes
    metrics = ['quality_change', 'latency_change', 'throughput_change', 'size_change']
    metric_labels = [quality_metric, 'Latency', 'Throughput', 'Model Size']
    plt.figure(figsize=(12, 6))
    for i, (metric, label) in enumerate(zip(metrics, metric_labels)):
        plt.bar(index + i*bar_width, top_5[metric], bar_width, label=label)
    plt.xlabel('Pipeline')
    plt.ylabel('Percentage Change (%)')
    plt.title('Percentage Change in Metrics by Pipeline')
    plt.xticks(index + 1.5*bar_width, [f"{row['device']}: {row['pipeline']}" for _, row in top_5.iterrows()], rotation=45, ha='right')
    plt.legend()
    plt.tight_layout()
    # plt.savefig('imgs/percent_change.png')
    plt.close()

    #  table data
    table_data = []
    # original row
    original_row_cpu = {
        'Pipeline': 'Original_CPU',
        quality_metric: f"{top_5[top_5.device=='CPU']['quality_init'].iloc[0]:.3f}",
        'CPU Latency (ms)': f"{df[df['device'] == 'CPU']['latency'].iloc[0]:.3f}" if len(df[df['device'] == 'CPU']['latency']) > 0 else np.NaN,
        'GPU Latency (ms)': np.NaN,
        'CPU Throughput (IPS)': f"{df[df['device'] == 'CPU']['throughput'].iloc[0]:.0f}" if len(df[df['device'] == 'CPU']['throughput']) > 0 else np.NaN,
        'GPU Throughput (IPS)': np.NaN,
        'Model Size (MB)': f"{top_5[top_5.device=='CPU']['size'].iloc[0]:.3f}"
    }
    original_row_gpu = {
        'Pipeline': 'Original_GPU',
        quality_metric: f"{top_5[top_5.device=='GPU']['quality_init'].iloc[0]:.3f}",
        'CPU Latency (ms)': np.NaN,
        'GPU Latency (ms)': f"{df[df['device'] == 'GPU']['latency'].iloc[0]:.3f}" if len(df[df['device'] == 'GPU']['latency']) > 0 else np.NaN,
        'CPU Throughput (IPS)': np.NaN,
        'GPU Throughput (IPS)': f"{df[df['device'] == 'GPU']['throughput'].iloc[0]:.0f}"if len(df[df['device'] == 'GPU']['throughput']) > 0 else np.NaN,
        'Model Size (MB)': f"{top_5[top_5.device=='GPU']['size'].iloc[0]:.3f}"
    }
    # table_data.append(original_row)


    for _, row in top_5.iterrows():
        table_row = {
            # 'Pipeline': f"{row['device']}: {row['pipeline']}",
            'Pipeline': f"{row['pipeline']}",
            'Device': row['device'],
            quality_metric: f"{row['quality_final']:.3f} / {row['quality_change']:+.1f}%",
            'CPU Latency (ms)': f"{row['latency']:.3f} / {row['latency_change']:+.1f}%" if row['device'] == 'CPU' else '∞',
            'GPU Latency (ms)': f"{row['latency']:.3f} / {row['latency_change']:+.1f}%" if row['device'] == 'GPU' else '∞',
            'CPU Throughput (IPS)': f"{row['throughput']:.0f} / {row['throughput_change']:+.1f}%" if row['device'] == 'CPU' else '∞',
            'GPU Throughput (IPS)': f"{row['throughput']:.0f} / {row['throughput_change']:+.1f}%" if row['device'] == 'GPU' else '∞',
            'Model Size (MB)': f"{row['size']:.3f} / {row['size_change']:+.1f}%"
        }
        table_data.append(table_row)

    table_df = pd.DataFrame(table_data)
    # table_df = table_df.groupby('Pipeline')[[
    #     quality_metric, 'CPU Latency (ms)', 'GPU Latency (ms)',
    #     'CPU Throughput (IPS)', 'GPU Throughput (IPS)', 'Model Size (MB)'
    #     ]].min().reset_index(drop=False)
    
    # original_df = pd.DataFrame(original_row, index=[0])

    original_df = pd.concat([
        pd.DataFrame(original_row_cpu, index=[0]),
        pd.DataFrame(original_row_gpu, index=[0]),
    ])
    table_df = pd.concat([original_df, table_df], axis=0).reset_index(drop=True)
    mapping = {'low-rank': 'LR', 'quant-static': 'QS', 'pruning': 'Pr', 'quant-qat': 'QAT', 'quant-dynamic': 'QD'}

    table_df['Pipeline'] = table_df['Pipeline'].apply(lambda x: ' - '.join([mapping.get(item, item) for item in x.split('_')]) if pd.notna(x) else x)
    table_df = table_df.replace('∞', '$\infty$')
    table_df.to_csv(f'tables/{tab_name}.csv', index=True)

    # latex table
    latex_content = """
\\begin{table*}[h]
\\centering
\\caption{Top 5 PEFT Pipelines}
\\label{tab:peft_pipelines}
\\resizebox{\\textwidth}{!}{%
\\begin{tabular}{lcccccc}
\\toprule
\\textbf{Pipeline} & \\textbf{""" + quality_metric + """} & \\textbf{CPU Latency (ms)} & \\textbf{GPU Latency (ms)} & \\textbf{CPU Throughput (IPS)} & \\textbf{GPU Throughput (IPS)} & \\textbf{Model Size (MB)} \\
\\midrule
"""
    for i, row in enumerate(table_data):
        escaped_row = {k: str(v).replace('%', '\%').replace('∞', '$\infty$') for k, v in row.items()}
        # latex_content += " & ".join(str(val) for val in row.values()) + " \\\\\n"
        latex_content += " & ".join(escaped_row.values()) + " \\\\\n"

        if i == 0:
            latex_content += "\\cmidrule{1-7}\n"

    latex_content += """\\bottomrule
\\end{tabular}
}
\\footnotesize
\\vspace{0.2cm}
\\emph{Note}: Values show final metric / percentage change from original. Best values are bold for non-quantized models and underlined for quantized models. Abbreviations: LR - Low-Rank Decomposition, Pr - Pruning, QAT - Quant-Aware Training, PDQ - Post-training Dynamic Quantization.
\\end{table*}
"""

    # Save LaTeX table
    with open(f'tables/{tab_name}_latex.txt', 'w') as f:
        f.write(latex_content)

    return table_df

In [2]:
tab = analyze_results(
    gpu_path="../results/ResNet18/CIFAR10",
    cpu_path="../results_cpu/ResNet18/CIFAR10",
    tab_name='cifar', 
    quality_metric='f1', 
    steps=3
    )

Top 6 Pipelines:
CPU - low-rank_quant-dynamic_pruning_quant-qat: f1 = 0.758 (-0.1%)
CPU - low-rank_quant-qat_pruning_quant-static: f1 = 0.747 (-1.6%)
CPU - low-rank_quant-static_pruning_quant-qat: f1 = 0.746 (-1.7%)
GPU - low-rank_quant-dynamic_pruning_quant-qat: f1 = 0.702 (-7.5%)
GPU - low-rank_quant-static_pruning_quant-qat: f1 = 0.660 (-13.0%)
GPU - low-rank_quant-static_pruning_quant-dynamic: f1 = 0.656 (-13.6%)


In [3]:
tab

,Pipeline,f1,CPU Latency (ms),GPU Latency (ms),CPU Throughput (IPS),GPU Throughput (IPS),Model Size (MB),Device
0,Original - CPU,0.759,0.004,NaN,1890,NaN,44.137,NaN
1,Original - GPU,0.759,NaN,1.901,NaN,17,43.349,NaN
2,LR - QD - Pr - QAT,0.702 / -7.5%,$\infty$,2.271 / +79.9%,$\infty$,15 / -42.0%,43.349 / +1.6%,GPU
3,LR - QS - Pr - QAT,0.660 / -13.0%,$\infty$,1.901 / +34.5%,$\infty$,17 / -24.4%,10.854 / -74.6%,GPU
4,LR - QS - Pr - QD,0.656 / -13.6%,$\infty$,1.873 / +34.9%,$\infty$,17 / -25.0%,10.861 / -74.5%,GPU
5,LR - QD - Pr - QAT,0.758 / -0.1%,0.005 / +9.6%,$\infty$,1436 / -22.8%,$\infty$,44.137 / +3.5%,CPU
6,LR - QAT - Pr - QS,0.747 / -1.6%,0.005 / +9.8%,$\infty$,1792 / -9.4%,$\infty$,44.137 / +3.5%,CPU
7,LR - QS - Pr - QAT,0.746 / -1.7%,0.004 / -4.9%,$\infty$,1890 / -6.7%,$\infty$,44.198 / +3.6%,CPU


## imagenette

In [4]:


gpu_path = "../results/ResNet18/Imagenette"
cpu_path = "../results_cpu/ResNet18/Imagenette"

# tab_name = 'imagenette'
tab = analyze_results(gpu_path=gpu_path, cpu_path=cpu_path, tab_name='imagenette', quality_metric='f1', steps=3)

Top 6 Pipelines:
CPU - low-rank_quant-qat_pruning_quant-static: f1 = 0.732 (+50.0%)
CPU - low-rank_quant-qat_pruning_quant-dynamic: f1 = 0.709 (+45.3%)
CPU - low-rank_quant-dynamic_pruning_quant-qat: f1 = 0.704 (+44.3%)
GPU - low-rank_quant-qat_pruning_quant-static: f1 = 0.632 (+35.3%)
GPU - quant-dynamic_pruning_quant-static_low-rank: f1 = 0.500 (+2172.7%)
GPU - low-rank_quant-qat_pruning_quant-dynamic: f1 = 0.498 (+6.6%)


In [5]:
tab

,Pipeline,f1,CPU Latency (ms),GPU Latency (ms),CPU Throughput (IPS),GPU Throughput (IPS),Model Size (MB),Device
0,Original - CPU,0.488,0.014,NaN,217,NaN,40.852,NaN
1,Original - GPU,0.467,NaN,2.306,NaN,16,9.960,NaN
2,LR - QAT - Pr - QS,0.632 / +35.3%,$\infty$,1.995 / +29.4%,$\infty$,18 / +64.6%,9.960 / -76.6%,GPU
3,QD - Pr - QS - LR,0.500 / +2172.7%,$\infty$,2.306 / +47.8%,$\infty$,16 / +44.1%,10.453 / -75.5%,GPU
4,LR - QAT - Pr - QD,0.498 / +6.6%,$\infty$,1.948 / +37.9%,$\infty$,18 / +61.5%,9.960 / -76.6%,GPU
5,LR - QAT - Pr - QS,0.732 / +50.0%,0.014 / +2.1%,$\infty$,208 / +35.8%,$\infty$,40.852 / -4.2%,CPU
6,LR - QAT - Pr - QD,0.709 / +45.3%,0.014 / +1.7%,$\infty$,214 / +35.0%,$\infty$,40.852 / -4.2%,CPU
7,LR - QD - Pr - QAT,0.704 / +44.3%,0.013 / +4.3%,$\infty$,226 / +38.3%,$\infty$,40.852 / -4.2%,CPU


## ApplienceEnergy

In [6]:


gpu_path = "../results/InceptionNet/AppliancesEnergy"
cpu_path = "../results_cpu/InceptionNet/AppliancesEnergy"

tab = analyze_results(gpu_path=gpu_path, cpu_path=cpu_path, tab_name='timseseries', quality_metric='rmse', steps=2)

Top 6 Pipelines:
GPU - pruning_quant-static_low-rank_quant-dynamic: rmse = 3.811 (+10.8%)
GPU - pruning_quant-static_low-rank_quant-qat: rmse = 3.817 (+10.9%)
GPU - pruning_quant-qat_low-rank_quant-dynamic: rmse = 3.841 (+11.6%)
CPU - pruning_quant-qat_low-rank_quant-static: rmse = 7.611 (+123.1%)
CPU - quant-qat_pruning_low-rank_quant-dynamic: rmse = 8.002 (+134.5%)
CPU - pruning_quant-dynamic_low-rank_quant-qat: rmse = 8.444 (+147.5%)


In [7]:
tab

,Pipeline,rmse,CPU Latency (ms),GPU Latency (ms),CPU Throughput (IPS),GPU Throughput (IPS),Model Size (MB),Device
0,Original - CPU,3.412,0.009,NaN,175,NaN,1.320,NaN
1,Original - GPU,3.441,NaN,4.345,NaN,2,0.923,NaN
2,Pr - QS - LR - QD,3.811 / +10.8%,$\infty$,3.852 / +30.4%,$\infty$,3 / -11.3%,0.923 / -68.8%,GPU
3,Pr - QS - LR - QAT,3.817 / +10.9%,$\infty$,3.538 / +15.8%,$\infty$,3 / -16.1%,0.923 / -68.8%,GPU
4,Pr - QAT - LR - QD,3.841 / +11.6%,$\infty$,3.664 / +22.8%,$\infty$,3 / -16.1%,0.923 / -68.8%,GPU
5,Pr - QAT - LR - QS,7.611 / +123.1%,0.006 / -7.5%,$\infty$,247 / -21.4%,$\infty$,1.320 / -55.3%,CPU
6,QAT - Pr - LR - QD,8.002 / +134.5%,0.006 / -1.9%,$\infty$,322 / -4.6%,$\infty$,0.437 / -85.2%,CPU
7,Pr - QD - LR - QAT,8.444 / +147.5%,0.007 / +22.7%,$\infty$,257 / -28.7%,$\infty$,1.694 / -42.7%,CPU
